# Following the code from here
https://huggingface.co/learn/nlp-course/en/chapter6/8?fw=pt

In [63]:
from datasets import concatenate_datasets, load_dataset
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
from transformers import PreTrainedTokenizerFast

In [10]:
bookcorpus = load_dataset("bookcorpus", split="train")
wiki = load_dataset("wikipedia", "20220301.en", split="train")
wiki = wiki.remove_columns([col for col in wiki.column_names if col != "text"])  # only keep the 'text' column
 
assert bookcorpus.features.type == wiki.features.type
raw_datasets = concatenate_datasets([bookcorpus, wiki])

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [45]:
def get_training_corpus():
    for i in range(0, len(raw_datasets), 1000):
        yield raw_datasets[i : i + 1000]["text"]

In [13]:
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [14]:
tokenizer

Tokenizer(version="1.0", truncation=None, padding=None, added_tokens=[], normalizer=None, pre_tokenizer=None, post_processor=None, decoder=None, model=WordPiece(unk_token="[UNK]", continuing_subword_prefix="##", max_input_chars_per_word=100, vocab={}))

## 1. Normalization (any cleanup of the text that is deemed necessary, such as removing spaces or accents, Unicode normalization, etc.)

In [32]:

tokenizer.normalizer = normalizers.BertNormalizer(
    clean_text = True, # remove all control characters and repleace repeating strings with a single one
    handle_chinese_chars=True, # place spaces around Chinese characters
    strip_accents=None, # whether to strip accents
    lowercase=True
)


In [33]:
# accents should be stripped since we are building an "uncased" tokenizer
# NFD = unicode normalizer
# shoudl also include 2x normalizers.Repalce Regex replacmeents
# tokenizer.normalizer = normalizers.Sequence(
#     [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
# )

In [34]:
tokenizer.normalizer.normalize_str("Héllò")

'hello'

In [35]:
tokenizer.normalizer.normalize_str("你好")

' 你  好 '

## 2. Pre-tokenization (splitting the input into words)

In [41]:
# splits on whitespace and any character that isn't a letter, digit, or underscore
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
# alternatively tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

In [42]:
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

[('Let', (0, 3)),
 ("'", (3, 4)),
 ('s', (4, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre', (14, 17)),
 ('-', (17, 18)),
 ('tokenizer', (18, 27)),
 ('.', (27, 28))]

## 3. Training
Run the input through the model (using the pre-tokenized words to produce a sequence of tokens)

In [46]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=30522, special_tokens=special_tokens)

In [47]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [48]:
# generate encoding. encoding contains fields
# ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, and overflowing
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['let', "'", 's', 'test', 'this', 'to', '##ke', '##n', '##ize', '##r', '.']


## 4. Post-processing
(adding the special tokens of the tokenizer, generating the attention mask and token type IDs)

In [49]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [50]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [51]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['[CLS]', 'let', "'", 's', 'test', 'this', 'to', '##ke', '##n', '##ize', '##r', '.', '[SEP]']


In [52]:
encoding = tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences.")
print(encoding.tokens)
print(encoding.type_ids)

['[CLS]', 'let', "'", 's', 'test', 'this', 'to', '##ke', '##n', '##ize', '##r', '.', '.', '.', '[SEP]', 'on', 'a', 'pair', 'of', 'sent', '##ences', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]


In [58]:
print("ids: ", encoding.ids)
print("special token mask: ", encoding.special_tokens_mask)

ids:  [2, 26173, 11, 61, 26723, 25217, 25052, 27426, 19872, 26155, 19861, 18, 18, 18, 3, 25072, 43, 29235, 25044, 26707, 28536, 18, 3]
offsets:  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]


## Decoder

In [60]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [61]:
tokenizer.decode(encoding.ids)

"let ' s test this tokenizer... on a pair of sentences."

In [62]:
tokenizer.save("tokenizer.json")

## Fast Tokenizer wrapper

In [64]:
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

# from transformers import BertTokenizerFast
# wrapped_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)